In [ ]:
from azureml.core import Workspace
ws = Workspace(subscription_id="364f4cb4-85da-4487-b721-c32eb218b904",
               resource_group="aguo921-rg",
               workspace_name="Image_Classification")
               

In [ ]:
from azureml.core.model import Model
model = Model.register(ws, model_name="image_classification", model_path="./output/my_model5/saved_model.pb")

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./echo_score.py",
)

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

In [ ]:
from azureml.core.runconfig import DockerConfiguration
DockerConfiguration(use_docker=True)

In [ ]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    compute_target,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)